# Use a federated learning strategy

Welcome to the next part of the federated learning tutorial. In previous parts of this tutorial, we introduced federated learning with PyTorch and Flower ([part 1](https://flower.dev/docs/framework/tutorial-get-started-with-flower-pytorch.html)).

In this notebook, we'll begin to customize the federated learning system we built in the introductory notebook (again, using [Flower](https://flower.dev/) and [PyTorch](https://pytorch.org/)).

> [Star Flower on GitHub](https://github.com/adap/flower) ⭐️ and join the Flower community on Slack to connect, ask questions, and get help: [Join Slack](https://flower.dev/join-slack) 🌼 We'd love to hear from you in the `#introductions` channel! And if anything is unclear, head over to the `#questions` channel.

Let's move beyond FedAvg with Flower strategies!

## Preparation

Before we begin with the actual code, let's make sure that we have everything we need.

### Installing dependencies

First, we install the necessary packages:

In [1]:
!pip install -q flwr[simulation] torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 MB 16.6 MB/s eta 0:00:00


Now that we have all dependencies installed, we can import everything we need for this tutorial:

In [2]:
from collections import OrderedDict
from typing import Dict, List, Optional, Tuple

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

import flwr as fl

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

Training on cpu using PyTorch 2.1.0+cu118 and Flower 1.6.0


It is possible to switch to a runtime that has GPU acceleration enabled (on Google Colab: `Runtime > Change runtime type > Hardware acclerator: GPU > Save`). Note, however, that Google Colab is not always able to offer GPU acceleration. If you see an error related to GPU availability in one of the following sections, consider switching back to CPU-based execution by setting `DEVICE = torch.device("cpu")`. If the runtime has GPU acceleration enabled, you should see the output `Training on cuda`, otherwise it'll say `Training on cpu`.

### Data loading

Let's now load the CIFAR-10 training and test set, partition them into ten smaller datasets (each split into training and validation set), and wrap everything in their own `DataLoader`. We introduce a new parameter `num_clients` which allows us to call `load_datasets` with different numbers of clients.

In [3]:
NUM_CLIENTS = 10

def load_datasets(num_clients: int):
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=True, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=True, transform=transform)

    # Split training set into `num_clients` partitions to simulate different local datasets
    partition_size = len(trainset) // num_clients
    lengths = [partition_size] * num_clients
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=32, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=32))
    testloader = DataLoader(testset, batch_size=32)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

100%|██████████| 170498071/170498071 [00:02<00:00, 78927789.86it/s]


Extracting ./dataset/cifar-10-python.tar.gz to ./dataset
Files already downloaded and verified


### Model training/evaluation

Let's continue with the usual model definition (including `set_parameters` and `get_parameters`), training and test functions:

In [4]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)


def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

### Flower client

To implement the Flower client, we (again) create a subclass of `flwr.client.NumPyClient` and implement the three methods `get_parameters`, `fit`, and `evaluate`. Here, we also pass the `cid` to the client and use it log additional details:

In [5]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=1)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

## Strategy customization

So far, everything should look familiar if you've worked through the introductory notebook. With that, we're ready to introduce a number of new features.

### Server-side parameter **initialization**

Flower, by default, initializes the global model by asking one random client for the initial parameters. In many cases, we want more control over parameter initialization though. Flower therefore allows you to directly pass the initial parameters to the Strategy:

In [6]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
)

# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-05 07:54:16,030 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-12-05 07:54:18,973	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-05 07:54:21,759 | app.py:213 | Flower VCE: Ray initialized with resources: {'memory': 7857539483.0, 'object_store_memory': 3928769740.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'memory': 7857539483.0, 'object_store_memory': 3928769740.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'CPU': 2.0}
INFO flwr 2023-12-05 07:54:21,769 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-05 07:

(DefaultActor pid=498) [Client 3] fit, config: {}
(DefaultActor pid=498) Epoch 1: train loss 0.06382150202989578, accuracy 0.23466666666666666
(DefaultActor pid=496) [Client 2] fit, config: {}
(DefaultActor pid=496) [Client 4] fit, config: {}


DEBUG flwr 2023-12-05 07:54:50,168 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:54:50,185 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 07:54:50,188 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(DefaultActor pid=496) [Client 2] evaluate, config: {}
(DefaultActor pid=496) Epoch 1: train loss 0.06378521025180817, accuracy 0.23755555555555555 [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)


DEBUG flwr 2023-12-05 07:54:55,220 | server.py:187 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:54:55,222 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 07:54:55,224 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

(DefaultActor pid=496) [Client 9] fit, config: {}
(DefaultActor pid=496) [Client 0] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=496) Epoch 1: train loss 0.0539792999625206, accuracy 0.36866666666666664
(DefaultActor pid=498) [Client 3] fit, config: {}
(DefaultActor pid=496) [Client 4] fit, config: {}


DEBUG flwr 2023-12-05 07:55:30,024 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-12-05 07:55:30,048 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=496) [Client 6] evaluate, config: {}
(DefaultActor pid=496) Epoch 1: train loss 0.053193312138319016, accuracy 0.3748888888888889 [repeated 2x across cluster]


DEBUG flwr 2023-12-05 07:55:35,412 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:55:35,416 | server.py:153 | FL finished in 73.57949386
INFO:flwr:FL finished in 73.57949386
INFO flwr 2023-12-05 07:55:35,419 | app.py:226 | app_fit: losses_distributed [(1, 0.061749436934789015), (2, 0.05572592043876648), (3, 0.05245797618230184)]
INFO:flwr:app_fit: losses_distributed [(1, 0.061749436934789015), (2, 0.05572592043876648), (3, 0.05245797618230184)]
INFO flwr 2023-12-05 07:55:35,422 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-05 07:55:35,423 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-05 07:55:35,425 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-12-05 07:55:35,426 | app.py:230 | app_fit: metrics_ce

History (loss, distributed):
	round 1: 0.061749436934789015
	round 2: 0.05572592043876648
	round 3: 0.05245797618230184

Passing `initial_parameters` to the `FedAvg` strategy prevents Flower from asking one of the clients for the initial parameters. If we look closely, we can see that the logs do not show any calls to the `FlowerClient.get_parameters` method.

### Starting with a customized strategy

We've seen the function `start_simulation` before. It accepts a number of arguments, amongst them the `client_fn` used to create `FlowerClient` instances, the number of clients to simulate `num_clients`, the number of rounds `num_rounds`, and the strategy.

The strategy encapsulates the federated learning approach/algorithm, for example, `FedAvg` or `FedAdagrad`. Let's try to use a different strategy this time:

In [7]:
# Create FedAdam strategy
strategy = fl.server.strategy.FedAdagrad(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-05 07:55:35,470 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-12-05 07:55:40,247	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-05 07:55:44,460 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7823241216.0, 'object_store_memory': 3911620608.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:__internal_head__': 1.0, 'node:172.28.0.12': 1.0, 'memory': 7823241216.0, 'object_store_memory': 3911620608.0}
INFO flwr 2023-12-05 07:55:44,472 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-05 07:

(DefaultActor pid=1054) [Client 3] fit, config: {}
(DefaultActor pid=1054) Epoch 1: train loss 0.06514626741409302, accuracy 0.21644444444444444
(DefaultActor pid=1051) [Client 7] fit, config: {}
(DefaultActor pid=1054) [Client 2] fit, config: {}


DEBUG flwr 2023-12-05 07:56:11,558 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:56:11,579 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 07:56:11,595 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


(DefaultActor pid=1054) Epoch 1: train loss 0.06479527801275253, accuracy 0.23466666666666666 [repeated 2x across cluster]
(DefaultActor pid=1054) [Client 1] evaluate, config: {}


DEBUG flwr 2023-12-05 07:56:18,147 | server.py:187 | evaluate_round 1 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:56:18,151 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 07:56:18,154 | server.py:222 | fit_round 2: strategy sampled 3 clients (out of 10)
DEBUG:flwr:fit_round 2: strategy sampled 3 clients (out of 10)


...

(DefaultActor pid=1054) [Client 8] fit, config: {}
(DefaultActor pid=1054) [Client 0] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=1054) Epoch 1: train loss 0.10471509397029877, accuracy 0.14266666666666666
(DefaultActor pid=1051) [Client 2] fit, config: {}
(DefaultActor pid=1054) [Client 6] fit, config: {}


DEBUG flwr 2023-12-05 07:56:50,280 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
DEBUG flwr 2023-12-05 07:56:50,296 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=1054) Epoch 1: train loss 0.10178906470537186, accuracy 0.1411111111111111 [repeated 2x across cluster]
(DefaultActor pid=1054) [Client 5] evaluate, config: {}


DEBUG flwr 2023-12-05 07:56:55,146 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:56:55,149 | server.py:153 | FL finished in 70.59022740699999
INFO:flwr:FL finished in 70.59022740699999
INFO flwr 2023-12-05 07:56:55,151 | app.py:226 | app_fit: losses_distributed [(1, 5.811706273396809), (2, 0.6774957205454508), (3, 0.07631497796376546)]
INFO:flwr:app_fit: losses_distributed [(1, 5.811706273396809), (2, 0.6774957205454508), (3, 0.07631497796376546)]
INFO flwr 2023-12-05 07:56:55,152 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-05 07:56:55,154 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-05 07:56:55,156 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-12-05 07:56:55,157 | app.py:230 | app_fit: metric

History (loss, distributed):
	round 1: 5.811706273396809
	round 2: 0.6774957205454508
	round 3: 0.07631497796376546

## Server-side parameter **evaluation**

Flower can evaluate the aggregated model on the server-side or on the client-side. Client-side and server-side evaluation are similar in some ways, but different in others.

**Centralized Evaluation** (or *server-side evaluation*) is conceptually simple: it works the same way that evaluation in centralized machine learning does. If there is a server-side dataset that can be used for evaluation purposes, then that's great. We can evaluate the newly aggregated model after each round of training without having to send the model to clients. We're also fortunate in the sense that our entire evaluation dataset is available at all times.

**Federated Evaluation** (or *client-side evaluation*) is more complex, but also more powerful: it doesn't require a centralized dataset and allows us to evaluate models over a larger set of data, which often yields more realistic evaluation results. In fact, many scenarios require us to use **Federated Evaluation** if we want to get representative evaluation results at all. But this power comes at a cost: once we start to evaluate on the client side, we should be aware that our evaluation dataset can change over consecutive rounds of learning if those clients are not always available. Moreover, the dataset held by each client can also change over consecutive rounds. This can lead to evaluation results that are not stable, so even if we would not change the model, we'd see our evaluation results fluctuate over consecutive rounds.

We've seen how federated evaluation works on the client side (i.e., by implementing the `evaluate` method in `FlowerClient`). Now let's see how we can evaluate aggregated model parameters on the server-side:

In [8]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy = test(net, valloader)
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy}

In [9]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,  # Pass the evaluation function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-05 07:56:55,210 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-12-05 07:56:59,811	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-05 07:57:02,695 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 7815453083.0, 'object_store_memory': 3907726540.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 7815453083.0, 'object_store_memory': 3907726540.0}
INFO flwr 2023-12-05 07:57:02,706 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-05 07:

Server-side evaluation loss 0.07392096185684204 / accuracy 0.08


(pid=1589) 2023-12-05 07:57:06.804716: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=1589) 2023-12-05 07:57:06.804799: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=1589) 2023-12-05 07:57:06.804835: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=1591) 2023-12-05 07:57:11.243024: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=1591) [Client 8] fit, config: {}
(DefaultActor pid=1589) Epoch 1: train loss 0.06463542580604553, accuracy 0.23666666666666666
(DefaultActor pid=1589) [Client 9] fit, config: {}
(DefaultActor pid=1589) [Client 1] fit, config: {}


DEBUG flwr 2023-12-05 07:57:30,581 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:57:30,595 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-12-05 07:57:30,819 | server.py:125 | fit progress: (1, 0.062154918432235716, {'accuracy': 0.29}, 27.218780734999996)
INFO:flwr:fit progress: (1, 0.062154918432235716, {'accuracy': 0.29}, 27.218780734999996)
DEBUG flwr 2023-12-05 07:57:30,821 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

(DefaultActor pid=1589) [Client 6] fit, config: {}
(DefaultActor pid=1589) [Client 0] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=1589) Epoch 1: train loss 0.05392584204673767, accuracy 0.35755555555555557
(DefaultActor pid=1591) [Client 3] fit, config: {}
(DefaultActor pid=1589) [Client 5] fit, config: {}


DEBUG flwr 2023-12-05 07:58:09,327 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:58:09,678 | server.py:125 | fit progress: (3, 0.0532948784828186, {'accuracy': 0.368}, 66.07747057799997)
INFO:flwr:fit progress: (3, 0.0532948784828186, {'accuracy': 0.368}, 66.07747057799997)
DEBUG flwr 2023-12-05 07:58:09,687 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


Server-side evaluation loss 0.0532948784828186 / accuracy 0.368
(DefaultActor pid=1589) [Client 6] evaluate, config: {}
(DefaultActor pid=1589) Epoch 1: train loss 0.053312718868255615, accuracy 0.3708888888888889 [repeated 2x across cluster]


DEBUG flwr 2023-12-05 07:58:15,343 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:58:15,346 | server.py:153 | FL finished in 71.74595746099999
INFO:flwr:FL finished in 71.74595746099999
INFO flwr 2023-12-05 07:58:15,348 | app.py:226 | app_fit: losses_distributed [(1, 0.061408273061116535), (2, 0.05572245566050212), (3, 0.051477984110514324)]
INFO:flwr:app_fit: losses_distributed [(1, 0.061408273061116535), (2, 0.05572245566050212), (3, 0.051477984110514324)]
INFO flwr 2023-12-05 07:58:15,349 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-05 07:58:15,353 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-05 07:58:15,355 | app.py:229 | app_fit: losses_centralized [(0, 0.07392096185684204), (1, 0.062154918432235716), (2, 0.05668581819534302), (3, 0.053294878482

History (loss, distributed):
	round 1: 0.061408273061116535
	round 2: 0.05572245566050212
	round 3: 0.051477984110514324
History (loss, centralized):
	round 0: 0.07392096185684204
	round 1: 0.062154918432235716
	round 2: 0.05668581819534302
	round 3: 0.0532948784828186
History (metrics, centralized):
{'accuracy': [(0, 0.08), (1, 0.29), (2, 0.334), (3, 0.368)]}

## Sending/receiving arbitrary values to/from clients

In some situations, we want to configure client-side execution (training, evaluation) from the server-side. One example for that is the server asking the clients to train for a certain number of local epochs. Flower provides a way to send configuration values from the server to the clients using a dictionary. Let's look at an example where the clients receive values from the server through the `config` parameter in `fit` (`config` is also available in `evaluate`). The `fit` method receives the configuration dictionary through the `config` parameter and can then read values from this dictionary. In this example, it reads `server_round` and `local_epochs` and uses those values to improve the logging and configure the number of local training epochs:

In [10]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        # Read values from config
        server_round = config["server_round"]
        local_epochs = config["local_epochs"]

        # Use values provided by the config
        print(f"[Client {self.cid}, round {server_round}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=local_epochs)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

So how can we  send this config dictionary from server to clients? The built-in Flower Strategies provide way to do this, and it works similarly to the way server-side evaluation works. We provide a function to the strategy, and the strategy calls this function for every round of federated learning:

In [11]:
def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        "local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

Next, we'll just pass this function to the FedAvg strategy before starting the simulation:

In [12]:
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=3,
    min_evaluate_clients=3,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    evaluate_fn=evaluate,
    on_fit_config_fn=fit_config,  # Pass the fit_config function
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-05 07:58:15,412 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-12-05 07:58:19,861	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-05 07:58:22,955 | app.py:213 | Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7813160142.0, 'object_store_memory': 3906580070.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7813160142.0, 'object_store_memory': 3906580070.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0}
INFO flwr 2023-12-05 07:58:22,962 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-05 07:

Server-side evaluation loss 0.07366375494003295 / accuracy 0.092


(pid=2135) 2023-12-05 07:58:28.544378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
(pid=2135) 2023-12-05 07:58:28.544454: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
(pid=2135) 2023-12-05 07:58:28.544503: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
(pid=2135) 2023-12-05 07:58:31.722942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


(DefaultActor pid=2135) [Client 9, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=2135) Epoch 1: train loss 0.06503409892320633, accuracy 0.21533333333333332
(DefaultActor pid=2136) [Client 7, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}
(DefaultActor pid=2136) [Client 1, round 1] fit, config: {'server_round': 1, 'local_epochs': 1}


DEBUG flwr 2023-12-05 07:58:51,221 | server.py:236 | fit_round 1 received 3 results and 0 failures
DEBUG:flwr:fit_round 1 received 3 results and 0 failures
WARNING flwr 2023-12-05 07:58:51,235 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
INFO flwr 2023-12-05 07:58:51,456 | server.py:125 | fit progress: (1, 0.06373861765861512, {'accuracy': 0.296}, 27.038496199000008)
INFO:flwr:fit progress: (1, 0.06373861765861512, {'accuracy': 0.296}, 27.038496199000008)
DEBUG flwr 2023-12-05 07:58:51,459 | server.py:173 | evaluate_round 1: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 1: strategy sampled 3 clients (out of 10)


...

(DefaultActor pid=2136) [Client 6, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=2136) [Client 8] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=2136) Epoch 1: train loss 0.051831673830747604, accuracy 0.39155555555555555
(DefaultActor pid=2135) [Client 2, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}
(DefaultActor pid=2136) Epoch 2: train loss 0.04897875711321831, accuracy 0.42488888888888887 [repeated 2x across cluster]
(DefaultActor pid=2136) [Client 3, round 3] fit, config: {'server_round': 3, 'local_epochs': 2}


DEBUG flwr 2023-12-05 07:59:48,731 | server.py:236 | fit_round 3 received 3 results and 0 failures
DEBUG:flwr:fit_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:59:48,964 | server.py:125 | fit progress: (3, 0.04856170272827148, {'accuracy': 0.454}, 84.546483121)
INFO:flwr:fit progress: (3, 0.04856170272827148, {'accuracy': 0.454}, 84.546483121)
DEBUG flwr 2023-12-05 07:59:48,967 | server.py:173 | evaluate_round 3: strategy sampled 3 clients (out of 10)
DEBUG:flwr:evaluate_round 3: strategy sampled 3 clients (out of 10)


(DefaultActor pid=2136) Epoch 2: train loss 0.04790085554122925, accuracy 0.4431111111111111 [repeated 3x across cluster]
Server-side evaluation loss 0.04856170272827148 / accuracy 0.454
(DefaultActor pid=2136) [Client 8] evaluate, config: {}


DEBUG flwr 2023-12-05 07:59:55,226 | server.py:187 | evaluate_round 3 received 3 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 3 results and 0 failures
INFO flwr 2023-12-05 07:59:55,229 | server.py:153 | FL finished in 90.81105869100003
INFO:flwr:FL finished in 90.81105869100003
INFO flwr 2023-12-05 07:59:55,230 | app.py:226 | app_fit: losses_distributed [(1, 0.06335735988616943), (2, 0.05135376906394958), (3, 0.04802222681045532)]
INFO:flwr:app_fit: losses_distributed [(1, 0.06335735988616943), (2, 0.05135376906394958), (3, 0.04802222681045532)]
INFO flwr 2023-12-05 07:59:55,234 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-05 07:59:55,236 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-05 07:59:55,239 | app.py:229 | app_fit: losses_centralized [(0, 0.07366375494003295), (1, 0.06373861765861512), (2, 0.0521415319442749), (3, 0.04856170272827148)

History (loss, distributed):
	round 1: 0.06335735988616943
	round 2: 0.05135376906394958
	round 3: 0.04802222681045532
History (loss, centralized):
	round 0: 0.07366375494003295
	round 1: 0.06373861765861512
	round 2: 0.0521415319442749
	round 3: 0.04856170272827148
History (metrics, centralized):
{'accuracy': [(0, 0.092), (1, 0.296), (2, 0.406), (3, 0.454)]}

As we can see, the client logs now include the current round of federated learning (which they read from the `config` dictionary). We can also configure local training to run for one epoch during the first and second round of federated learning, and then for two epochs during the third round.

Clients can also return arbitrary values to the server. To do so, they return a dictionary from `fit` and/or `evaluate`. We have seen and used this concept throughout this notebook without mentioning it explicitly: our `FlowerClient` returns a dictionary containing a custom key/value pair as the third return value in `evaluate`.

## Scaling federated learning

As a last step in this notebook, let's see how we can use Flower to experiment with a large number of clients.

In [13]:
NUM_CLIENTS = 1000

trainloaders, valloaders, testloader = load_datasets(NUM_CLIENTS)

Files already downloaded and verified
Files already downloaded and verified


We now have 1000 partitions, each holding 45 training and 5 validation examples. Given that the number of training examples on each client is quite small, we should probably train the model a bit longer, so we configure the clients to perform 3 local training epochs. We should also adjust the fraction of clients selected for training during each round (we don't want all 1000 clients participating in every round), so we adjust `fraction_fit` to `0.05`, which means that only 5% of available clients (so 50 clients) will be selected for training each round:


In [14]:
def fit_config(server_round: int):
    config = {
        "server_round": server_round,
        "local_epochs": 3,
    }
    return config


strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.025,  # Train on 25 clients (each round)
    fraction_evaluate=0.05,  # Evaluate on 50 clients (each round)
    min_fit_clients=20,
    min_evaluate_clients=40,
    min_available_clients=NUM_CLIENTS,
    initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net())),
    on_fit_config_fn=fit_config,
)

fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
)

INFO flwr 2023-12-05 07:59:57,265 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=3, round_timeout=None)
2023-12-05 08:00:01,760	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2023-12-05 08:00:04,709 | app.py:213 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 7811764224.0, 'object_store_memory': 3905882112.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'memory': 7811764224.0, 'object_store_memory': 3905882112.0, 'CPU': 2.0}
INFO flwr 2023-12-05 08:00:04,720 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2023-12-05 08:

(DefaultActor pid=2769) [Client 383, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=2769) Epoch 1: train loss 0.10201366990804672, accuracy 0.2
(DefaultActor pid=2769) Epoch 2: train loss 0.10125135630369186, accuracy 0.2
(DefaultActor pid=2768) [Client 192, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 3x across cluster]
(DefaultActor pid=2769) Epoch 3: train loss 0.10104789584875107, accuracy 0.17777777777777778 [repeated 7x across cluster]
(DefaultActor pid=2769) [Client 285, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=2769) Epoch 3: train loss 0.10056347399950027, accuracy 0.2222222222222222 [repeated 6x across cluster]
(DefaultActor pid=2768) [Client 211, round 1] fit, config: {'server_round': 1, 'local_epochs': 3}
(DefaultActor pid=2768) Epoch 3: train loss 0.10067634284496307, accuracy 0.17777777777777778 [repeated 9x across cluster]
(DefaultActor pid=2769) [Client 547, round 1] fit, conf

DEBUG flwr 2023-12-05 08:01:04,778 | server.py:236 | fit_round 1 received 25 results and 0 failures
DEBUG:flwr:fit_round 1 received 25 results and 0 failures
WARNING flwr 2023-12-05 08:01:04,844 | fedavg.py:242 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 08:01:04,846 | server.py:173 | evaluate_round 1: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 1: strategy sampled 50 clients (out of 1000)


(DefaultActor pid=2768) [Client 881] evaluate, config: {}
(DefaultActor pid=2768) Epoch 3: train loss 0.10095063596963882, accuracy 0.28888888888888886 [repeated 11x across cluster]
(DefaultActor pid=2768) [Client 593, round 1] fit, config: {'server_round': 1, 'local_epochs': 3} [repeated 3x across cluster]
(DefaultActor pid=2769) [Client 488] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2769) [Client 342] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 837] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2768) [Client 926] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2769) [Client 798] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2768) [Client 884] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2768) [Client 749] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 734] evaluate, config: {} [repeated 3x across clus

DEBUG flwr 2023-12-05 08:02:30,117 | server.py:187 | evaluate_round 1 received 50 results and 0 failures
DEBUG:flwr:evaluate_round 1 received 50 results and 0 failures
WARNING flwr 2023-12-05 08:02:30,119 | fedavg.py:273 | No evaluate_metrics_aggregation_fn provided
DEBUG flwr 2023-12-05 08:02:30,122 | server.py:222 | fit_round 2: strategy sampled 25 clients (out of 1000)
DEBUG:flwr:fit_round 2: strategy sampled 25 clients (out of 1000)


...

(DefaultActor pid=2769) [Client 929, round 3] fit, config: {'server_round': 3, 'local_epochs': 3}
(DefaultActor pid=2769) [Client 406] evaluate, config: {} [repeated 2x across cluster]
(DefaultActor pid=2769) Epoch 1: train loss 0.10137572884559631, accuracy 0.13333333333333333
(DefaultActor pid=2769) Epoch 2: train loss 0.0996728464961052, accuracy 0.24444444444444444
(DefaultActor pid=2769) Epoch 3: train loss 0.09805518388748169, accuracy 0.24444444444444444
(DefaultActor pid=2768) [Client 938, round 3] fit, config: {'server_round': 3, 'local_epochs': 3} [repeated 3x across cluster]
(DefaultActor pid=2769) Epoch 3: train loss 0.09310071915388107, accuracy 0.3333333333333333 [repeated 6x across cluster]
(DefaultActor pid=2768) [Client 484, round 3] fit, config: {'server_round': 3, 'local_epochs': 3} [repeated 4x across cluster]
(DefaultActor pid=2769) Epoch 3: train loss 0.09535480290651321, accuracy 0.3111111111111111 [repeated 12x across cluster]
(DefaultActor pid=2768) [Client 324

DEBUG flwr 2023-12-05 08:05:30,809 | server.py:236 | fit_round 3 received 25 results and 0 failures
DEBUG:flwr:fit_round 3 received 25 results and 0 failures
DEBUG flwr 2023-12-05 08:05:30,871 | server.py:173 | evaluate_round 3: strategy sampled 50 clients (out of 1000)
DEBUG:flwr:evaluate_round 3: strategy sampled 50 clients (out of 1000)


(DefaultActor pid=2768) [Client 452, round 3] fit, config: {'server_round': 3, 'local_epochs': 3} [repeated 4x across cluster]
(DefaultActor pid=2768) Epoch 2: train loss 0.0979778915643692, accuracy 0.24444444444444444 [repeated 14x across cluster]
(DefaultActor pid=2768) [Client 633] evaluate, config: {}
(DefaultActor pid=2768) Epoch 3: train loss 0.09684359282255173, accuracy 0.24444444444444444
(DefaultActor pid=2769) [Client 203] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2769) [Client 5] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 478] evaluate, config: {} [repeated 3x across cluster]
(DefaultActor pid=2768) [Client 166] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 796] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 529] evaluate, config: {} [repeated 4x across cluster]
(DefaultActor pid=2768) [Client 153] evaluate, config: {} [repeated 4x across 

DEBUG flwr 2023-12-05 08:06:58,049 | server.py:187 | evaluate_round 3 received 50 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 50 results and 0 failures
INFO flwr 2023-12-05 08:06:58,051 | server.py:153 | FL finished in 413.22985575699994
INFO:flwr:FL finished in 413.22985575699994
INFO flwr 2023-12-05 08:06:58,054 | app.py:226 | app_fit: losses_distributed [(1, 0.4599130268096923), (2, 0.45515338897705077), (3, 0.44621464633941643)]
INFO:flwr:app_fit: losses_distributed [(1, 0.4599130268096923), (2, 0.45515338897705077), (3, 0.44621464633941643)]
INFO flwr 2023-12-05 08:06:58,059 | app.py:227 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-12-05 08:06:58,061 | app.py:228 | app_fit: metrics_distributed {}
INFO:flwr:app_fit: metrics_distributed {}
INFO flwr 2023-12-05 08:06:58,062 | app.py:229 | app_fit: losses_centralized []
INFO:flwr:app_fit: losses_centralized []
INFO flwr 2023-12-05 08:06:58,064 | app.py:230 | app_fit

History (loss, distributed):
	round 1: 0.4599130268096923
	round 2: 0.45515338897705077
	round 3: 0.44621464633941643

## Recap

In this notebook, we've seen how we can gradually enhance our system by customizing the strategy, initializing parameters on the server side, choosing a different strategy, and evaluating models on the server-side. That's quite a bit of flexibility with so little code, right?

In the later sections, we've seen how we can communicate arbitrary values between server and clients to fully customize client-side execution. With that capability, we built a large-scale Federated Learning simulation using the Flower Virtual Client Engine and ran an experiment involving 1000 clients in the same workload - all in a Jupyter Notebook!

## Next steps

Before you continue, make sure to join the Flower community on Slack: [Join Slack](https://flower.dev/join-slack/)

There's a dedicated `#questions` channel if you need help, but we'd also love to hear who you are in `#introductions`!

The [Flower Federated Learning Tutorial - Part 3](https://flower.dev/docs/framework/tutorial-build-a-strategy-from-scratch-pytorch.html) shows how to build a fully custom `Strategy` from scratch.